In [1]:
import base64
import collections
import copy
import datetime
import itertools
import json
import math
import os
import operator
import pickle
import random
import re
import shutil
import sys
import time

import cv2
import Image
import keras
from keras import *
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL as pil
import pylab
import scipy
import seaborn as sns
import sklearn
from sklearn import *
import statsmodels as sm
import tensorflow as tf

np.random.seed(1337)

%matplotlib inline

sns.set(font_scale=1.3)
mpl.rcParams['figure.figsize'] = 20, 12
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

linewidth = 1.0
dotsize = 15

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/usr/lo

In [6]:
base_dir = '/media/ntfs/data/invasive_species/'
input_dir = os.path.join(base_dir, 'input')
tmp_dir = os.path.join(base_dir, 'tmp')
output_dir = os.path.join(base_dir, 'output')

In [ ]:
train_dir = os.path.join(tmp_dir, 'train')
os.mkdir(train_dir)

val_dir = os.path.join(tmp_dir, 'val')
os.mkdir(val_dir)

test_dir = os.path.join(tmp_dir, 'test')
os.mkdir(test_dir)

In [ ]:
train_pos_dir = os.path.join(train_dir, 'pos')
os.mkdir(train_pos_dir)

train_neg_dir = os.path.join(train_dir, 'neg')
os.mkdir(train_neg_dir)

val_pos_dir = os.path.join(val_dir, 'pos')
os.mkdir(val_pos_dir)

val_neg_dir = os.path.join(val_dir, 'neg')
os.mkdir(val_neg_dir)

In [18]:
d = pd.read_csv(input_dir + '/train_labels.csv')
d = train.sample(frac=1.0)

print len(d)
print '-' * 30
print d.head(10)
print '-' * 30
print d.invasive.value_counts()

2295
------------------------------
      name  invasive
176    177         1
133    134         1
636    637         1
690    691         1
1871  1872         1
511    512         1
5        6         0
1194  1195         1
1846  1847         0
490    491         1
------------------------------
1    1448
0     847
Name: invasive, dtype: int64


In [22]:
train_frac = 0.8
train_cnt = int(train_frac * len(d))

train = d[:train_cnt]
train_pos_names = train[train.invasive == 1].name.tolist()
train_neg_names = train[train.invasive == 0].name.tolist()

val = d[train_cnt:]
val_pos_names = val[val.invasive == 1].name.tolist()
val_neg_names = val[val.invasive == 0].name.tolist()

In [34]:
dirs_names = [
    (train_pos_dir, train_pos_names),
    (train_neg_dir, train_neg_names),
    (val_pos_dir, val_pos_names),
    (val_neg_dir, val_neg_names),
]

for p in dirs_names:
    for n in p[1]:
        shutil.copyfile(input_dir + '/train/{0}.jpg'.format(n), p[0] + '/{0}.jpg'.format(n))